<a href="https://colab.research.google.com/github/ShlokArora2709/Data-science/blob/main/Deep_Learning/model_training_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'food-classification-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4391402%2F7589488%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240228%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240228T055811Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3d6e6da22ed4133ae43646e6b54e71cb38cc5b4d877b90bfb291d6b066bb404eb6c520574d5d713bd564012a43df01aa0492dc7c8aabb7ae4843a4d2d14756c704a595f392326fd304bdea774152f496234850d7c42617b86bb462e89cc4b6ec77e1fa2902e94cf4858e34ea8424ce5b4e9dc43289edceb21c814340ecf055b979e3d3790ffa6b23c76fef7d47790b0a5998a362d781ebb14d5caa944926e596443d67bca1d22c9acfae59c0d8d32f8e9359399a5f103456b69e27de5ee6e753c5ec6117817bfa8b99882df258eadc2c61aa9931be3a2cb8520bb0b1a193235a4dd1e8bdb3fda535edd5a842a52715cde8c44ce0843d61fdd7f4020c5bb53932'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import numpy as np
from keras.applications import VGG16,InceptionResNetV2
from keras.models import Model
from keras.layers import LSTM, Dense, Dropout, Flatten
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt

import os, random, pathlib, warnings, itertools, math
warnings.filterwarnings("ignore")

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K

from keras import regularizers
from tensorflow.keras import models,Sequential,layers
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dense, Dropout,LeakyReLU
from tensorflow.keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
import cv2

In [ ]:
IMAGE_SIZE=[256,256]
train_datagen = image.ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=40,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest')

test_datagen = image.ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(
    '/kaggle/input/food-classification-dataset/dataset/train',
    target_size = IMAGE_SIZE,
    batch_size = 64,
    class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(
    '/kaggle/input/food-classification-dataset/dataset/test',
    target_size = IMAGE_SIZE,
    batch_size = 64,
    class_mode = 'categorical')


In [ ]:
test_set.class_indices

In [ ]:
training_set.class_indices

In [ ]:
base= InceptionResNetV2(weights='imagenet', include_top=False)

for layer in base.layers:
    layer.trainable = True
    if isinstance(layer, tf.keras.layers.Conv2D):
        layer.kernel_regularizer = kernel_regularizer=regularizers.L2(0.005)

# Find the index of the layer where 'mixed9' is present
block8_1_mixed = base.layers.index(next(layer for layer in base.layers if layer.name == 'block8_1_mixed'))

# Set trainable to False for layers before 'mixed9'
for layer in base.layers[:block8_1_mixed]:
    layer.trainable = False

for layer in base.layers:
    print(layer.name, layer.trainable)

In [ ]:
I = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in I.layers:
    layer.trainable = True
    if isinstance(layer, tf.keras.layers.Conv2D):
        layer.kernel_regularizer = kernel_regularizer=regularizers.L1L2(0.1)

# Find the index of the layer where 'mixed9' is present
mixed9_index = I.layers.index(next(layer for layer in I.layers if layer.name == 'mixed9'))

# Set trainable to False for layers before 'mixed9'
for layer in I.layers[:mixed9_index]:
    layer.trainable = False

# Print the trainable status of each layer
for layer in I.layers:
    print(layer.name, layer.trainable)

I.summary()


In [ ]:
model = Sequential()
model.add(base)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(36,activation='softmax'))

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
model.summary()

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer=Adam(),
  metrics=['accuracy']
)
#keras.optimizers.RMSprop(lr=1e-5)

In [ ]:
his = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=7)

In [ ]:
model.save("InceptionResNet_DataV5_96.h5")

In [ ]:
category = {
    0: 'Bacon',
    1: 'Banana',
    2: 'Bean',
    3: 'Bitter_gourd',
    4: 'Bottle_gourd',
    5: 'Bread',
    6: 'Brinjal',
    7: 'Broccoli',
    8: 'Butter',
    9: 'Cabbage',
    10: 'Capsicum',
    11: 'Carrots',
    12: 'Cauliflower',
    13: 'Cheese',
    14: 'Chicken',
    15: 'Chilli',
    16: 'Corn',
    17: 'Cucumber',
    18: 'Eggplant',
    19: 'Eggs',
    20: 'Fish',
    21: 'Garlic',
    22: 'Ginger',
    23: 'Lettuce',
    24: 'Meat',
    25: 'Milk',
    26: 'Noodles-pasta',
    27: 'Onions',
    28: 'Papaya',
    29: 'Potatoes',
    30: 'Pumpkin',
    31: 'Radish',
    32: 'Rice',
    33: 'Spinach',
    34: 'Tomato',
    35: 'Yogurt'
}


In [ ]:
import json
with open("food_map.json", "w") as json_file:
    json.dump(category, json_file, indent=4)

In [ ]:

def predict_image(filename,model):
    img_ = image.load_img(filename, target_size=IMAGE_SIZE)
    img_array = image.img_to_array(img_)
    img_processed = np.expand_dims(img_array, axis=0)
    img_processed /= 255.

    prediction = model.predict(img_processed)
    index = np.argmax(prediction)

    plt.title("Prediction - {}".format(category[index]))
    plt.imshow(img_array)

In [ ]:
predict_image('/kaggle/input/food-classification-dataset/dataset/test/Corn/Corn12.jpg',model)

In [ ]:
predict_image('/kaggle/input/food-classification-dataset/dataset/test/Broccoli/1214.jpg',model)

In [ ]:
predict_image('/kaggle/input/food-classification-dataset/dataset/test/Yogurt/yogurt.123.jpg',model)

In [ ]:
predict_image('/kaggle/input/food-classification-dataset/dataset/test/Milk/Milk_44.jpg',model)

In [ ]:
predict_image('/kaggle/input/food-classification-dataset/dataset/test/Meat/ad.14.png',model)

In [ ]:
predict_image('/kaggle/input/food-classification-dataset/dataset/test/Butter/butter.154.jpg',model)

In [ ]:
predict_image('/kaggle/input/food-classification-dataset/dataset/test/Cheese/cheese.121.jpg',model)

In [ ]:
predict_image('/kaggle/input/food-classification-dataset/dataset/test/Carrots/1312.jpg',model)

In [ ]:
pred=model.predict_generator(test_set)
y_pred = np.argmax(pred, axis=1)
y_true = test_set.classes

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

conf_matrix = confusion_matrix(y_true, y_pred)

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='YlOrBr')

# Add labels and title
plt.xlabel('Predicted labels')
plt.ylabel('Actual labels')
plt.title('Confusion Matrix')

# Display the heatmap
plt.show()